In [0]:
##Q1####################################################
import sqlite3

dbname="test.db"

### SQL Commands
salesman = """
CREATE TABLE salesman(
    salesman_id NUM AUTO INCREMENT,
    name TEXT,
    city TEXT,
    commission NUM,
    PRIMARY KEY(salesman_id)
    );
"""
customer = """
CREATE TABLE customer(
    customer_id NUM AUTO INCREMENT,
    cust_name TEXT,
    city TEXT,
    grade NUM,
    salesman_id NUM,
    PRIMARY KEY(customer_id),
    FOREIGN KEY(salesman_id) REFERENCES salesman(salesman_id)
    );
"""
orders = """
CREATE TABLE orders(
    ord_no NUM AUTO INCREMENT,
    purch_amt NUM,
    ord_date TEXT,
    customer_id NUM,
    salesman_id NUM,
    PRIMARY KEY(ord_no),
    FOREIGN KEY(customer_id) REFERENCES customer(customer_id),
    FOREIGN KEY(salesman_id) REFERENCES salesman(salesman_id)
    );
"""
def createTables(conn):
    conn.executescript(salesman+customer+orders)
    conn.commit()
    
conn = sqlite3.connect(dbname)
createTables(conn)
conn.close()

In [0]:
###Q2#################################################################
def properSpacing(text):
    i = 0
    #Usage of while loop as length of text changes on runtime
    while i < len(text): 
        # Ignore the first letter and 
        # check if the letter is in CAPS with ASCII codes
        if (i != 0 and 65 <= ord(text[i]) <= 65 + 26):
            # Use String slicing to insert space
            text = text[:i]+" "+text[i:]
            i += 2 #Skip spacing AND the current letter
        else:
            i += 1 #Move on to next letter
    return text

def customerDataProcessing(line):
    for i in range(len(line)):
        line[i] = properSpacing(line[i])
    return line

import csv
def importData(conn):
    with open("salesman.txt") as f:
        reader = csv.reader(f)
        for line in reader:
            if line[0].isdigit():
                #print(tuple(line))
                conn.execute("INSERT INTO salesman(salesman_id,name,city,commission) VALUES (?,?,?,?)",
                            tuple(line))
    with open("customer.txt") as f:
        reader = csv.reader(f)
        for line in reader:
            if line[0].isdigit():
                #print(tuple(line))
                conn.execute("INSERT INTO customer(customer_id,cust_name,city,grade,salesman_id) VALUES (?,?,?,?,?)",
                            tuple(customerDataProcessing(line)))
    with open("orders.txt") as f:
        reader = csv.reader(f)
        for line in reader:
            if line[0].isdigit():
                
                conn.execute("INSERT INTO orders(ord_no, purch_amt, ord_date, customer_id, salesman_id) VALUES(?,?,?,?,?)",
                            tuple(line))
    conn.commit()

conn = sqlite3.connect(dbname)
importData(conn)     
conn.close()

In [0]:
###Q3#####################################################################
menuOptions="""\
Options
1. List of orders by order amount
2. List of salesman working for which customer
3. List order details
4. List of customer orders
5. Exit\
"""
def menu(conn):
    while True:
        print(menuOptions)
        option = int(input("Enter option:"))
        if option == 1:
            minAmt = int(input("Please input in the minimum order amount:" ))
            maxAmt = int(input("Please input in the maximum  order amount: "))
            cursor = conn.execute("SELECT ord_no,purch_amt,customer.cust_name,city FROM orders  LEFT OUTER JOIN customer ON orders.customer_id=customer.customer_id WHERE purch_amt>=? AND purch_amt<=? ORDER BY ord_no ASC",
                                 (minAmt,maxAmt))
            print("\nList of orders between",minAmt,"and",str(maxAmt)+":\n")
            header = ["ord_no","purch_amt","cust_name","city"]
            for j in header:
                print(f"{j:<15} ",end="")
            print()
            for i in cursor:
                for j in i:
                    print(f"{j:<15} ",end="")
                print()    
            print() # New line for proper spacing
        elif option == 2:
            name = input("Name of salesman: ")
            cursor = conn.execute("SELECT cust_name,customer.city,salesman.name,commission FROM customer LEFT OUTER JOIN salesman ON customer.salesman_id=salesman.salesman_id WHERE salesman.name=?",
                                 (name,))
            print("")
            header = ["Customer Name","city","Salesman","commission"]
            for j in header:
                print(f"{j:<15} ",end="")
            print()
            for i in cursor:
                for j in i:
                    print(f"{j:<15} ",end="")
                print()
            print() #Newline for Proper Spacing

        elif option == 3:
            print() #Newline for Proper Spacing
            cursor = conn.execute("SELECT ord_no,ord_date,purch_amt,cust_name,grade,name,commission  FROM (orders INNER JOIN customer ON orders.customer_id=customer.customer_id) INNER JOIN salesman ON orders.salesman_id=salesman.salesman_id ")
            header = ["ord_no","ord_date","purch_amt","Customer Name","grade","Salesman","commission"]
            for j in header:
                print(f"{j:<15} ",end="")
            print()
            for i in cursor:
                for j in i:
                    print(f"{j:<15} ",end="")
                print()
            print() #Newline for Proper Spacing
        
        elif option == 4:
            print() #Newline for Proper Spacing
            cursor = conn.execute("SELECT cust_name,customer.city,ord_no,ord_date,purch_amt FROM orders LEFT OUTER JOIN customer ON orders.customer_id=customer.customer_id ORDER BY ord_date ASC")
            header = ["cust_name","city","ord_no","ord_date","Order Amount"]
            for j in header:
                print(f"{j:<15} ",end="")
            print()
            for i in cursor:
                for j in i:
                    print(f"{j:<15} ",end="")
                print()#Newline
            print() #Newline for Proper Spacing

        elif option == 5:
            return

conn = sqlite3.connect(dbname)
menu(conn)     
conn.close()

Options
1. List of orders by order amount
2. List of salesman working for which customer
3. List order details
4. List of customer orders
5. Exit
Enter option:1
Please input in the minimum order amount:500
Please input in the maximum  order amount: 2000

List of orders between 500 and 2000:

ord_no          purch_amt       cust_name       city            
70007           948.5           Graham Zusi     California      
70010           1983.43         Fabian Johnson  Paris           

Options
1. List of orders by order amount
2. List of salesman working for which customer
3. List order details
4. List of customer orders
5. Exit
Enter option:2
Name of salesman: James Hoog

Customer Name   city            Salesman        commission      
Nick Rimando    New York        James Hoog      0.15            
Brad Davis      New York        James Hoog      0.15            

Options
1. List of orders by order amount
2. List of salesman working for which customer
3. List order details
4. List of cu

In [0]:
conn.close()
